In [1]:
import pandas as pd
import numpy as np
from funciones import *
import geopandas as gpd
import matplotlib.pyplot as plt
import datetime as dt

In [2]:
from shapely.geometry import Point, LineString, Polygon

In [3]:
barrios = gpd.read_file(r'..\data\geo_data\Barrios\Barrios.shp')

In [4]:
estacionex = gpd.read_file(r'..\data\geo_data\estaciones.geojson')

In [5]:
carriles = gpd.read_file(r'..\data\geo_data\Infraestructura_Ciclista\Infraestructura_Ciclista.shp')

In [6]:
#carriles = carriles.to_crs(4326) 

In [7]:
estacionex = estacionex.to_crs(25830)

In [8]:
estacionex

,id,number,name,longitude,latitude,address,total_bases,geometry
0,1,1a,Puerta del Sol A,-3.7024255,40.4172137,Puerta del Sol nº 1,30,POINT (440407.435 4474302.375)
1,2,1b,Puerta del Sol B,-3.7024207,40.4170009,Puerta del Sol nº 1,30,POINT (440407.654 4474278.751)
2,3,2,Miguel Moya,-3.7058415,40.4205886,Calle Miguel Moya nº 1,24,POINT (440120.617 4474679.297)
3,4,3,Plaza Conde Suchil,-3.7069171,40.4302937,Plaza del Conde del Valle de Súchil nº 3,18,POINT (440037.988 4475757.290)
4,5,4,Malasaña,-3.7025875,40.4285524,Calle Manuela Malasaña nº 5,24,POINT (440403.699 4475561.076)
...,...,...,...,...,...,...,...,...
261,266,258,Ciudad Universitaria 1,-3.72699,40.44375,Avenida de la Complutense (Metro Ciudad Univer...,24,POINT (438347.647 4477264.758)
262,267,259,Ciudad Universitaria 2,-3.72693,40.44342,Avenida de la Complutense (Metro Ciudad Univer...,24,POINT (438352.434 4477228.086)
263,268,260,Facultad Biología,-3.72731,40.44912,"Jose Antonio Novais, frente al 12",24,POINT (438325.418 4477861.052)
264,269,261,Facultad Derecho,-3.72937,40.45109,"Avenida Complutense, 23",24,POINT (438152.538 4478081.164)


In [9]:
carriles.head()

,TIPO_VIA,ID_VIA,TIPOLOGÍA,DIRECCIÓN,SENTIDO,PENDIENTE,PENDIENTE_,COD_TIPOLO,COD_TIPO_V,Shape_STLe,d_COD_TIPO,d_COD_TI_1,geometry
0,CICLOCARRIL,0.0,VÍA USO COMPARTIDO,Avenida de Valladolid,None,None,0.0,5,19,1102.498490,VÍA USO COMPARTIDO,"VÍA USO COMPARTIDO, CICLOCARRIL","LINESTRING Z (437719.596 4476171.279 0.000, 43..."
1,CICLOCARRIL,1.0,VÍA USO COMPARTIDO,Avenida de Valladolid,None,None,0.0,5,19,1105.274193,VÍA USO COMPARTIDO,"VÍA USO COMPARTIDO, CICLOCARRIL","LINESTRING Z (437725.280 4476179.411 0.000, 43..."
2,CICLOCARRIL,2.0,VÍA USO COMPARTIDO,Paseo de la Virgen del Puerto,None,None,0.0,5,19,1044.939833,VÍA USO COMPARTIDO,"VÍA USO COMPARTIDO, CICLOCARRIL","LINESTRING Z (438833.498 4473930.951 0.000, 43..."
3,CICLOCARRIL,3.0,VÍA USO COMPARTIDO,Calle de Juan Duque,None,None,0.0,5,19,567.611720,VÍA USO COMPARTIDO,"VÍA USO COMPARTIDO, CICLOCARRIL","LINESTRING Z (438943.335 4473376.361 0.000, 43..."
4,CICLOCARRIL,4.0,VÍA USO COMPARTIDO,Avenida de la Albufera,None,None,0.0,5,19,2735.758322,VÍA USO COMPARTIDO,"VÍA USO COMPARTIDO, CICLOCARRIL","LINESTRING Z (445628.922 4470978.234 0.000, 44..."


In [10]:
carriles.TIPOLOGÍA.value_counts()

VÍA USO COMPARTIDO       653
VÍA EXCLUSIVA BICI       433
ANILLO VERDE CICLISTA     50
VÍA PREFERENTE BICI        9
GIROS Y SENTIDOS           6
Name: TIPOLOGÍA, dtype: int64

In [11]:
carriles.TIPOLOGÍA = carriles.TIPOLOGÍA.str.replace('VÍA EXCLUSIVA BICI', 'BICI')
carriles.TIPOLOGÍA = carriles.TIPOLOGÍA.str.replace('ANILLO VERDE CICLISTA', 'BICI')
carriles.TIPOLOGÍA = carriles.TIPOLOGÍA.str.replace('VÍA PREFERENTE BICI', 'BICI')
carriles.TIPOLOGÍA = carriles.TIPOLOGÍA.str.replace('GIROS Y SENTIDOS', 'BICI')

In [12]:
carriles.TIPOLOGÍA.value_counts()

VÍA USO COMPARTIDO    653
BICI                  498
Name: TIPOLOGÍA, dtype: int64

In [13]:
carriles_1 = carriles[['TIPOLOGÍA', 'geometry']].dissolve(by='TIPOLOGÍA')

In [14]:
carriles_1

,geometry
TIPOLOGÍA,
BICI,MULTILINESTRING Z ((448462.161 4469869.424 0.0...
VÍA USO COMPARTIDO,MULTILINESTRING Z ((437719.596 4476171.279 0.0...


In [15]:
buf = estacionex.buffer(500)

In [16]:
buf1 = buf.to_frame().join(estacionex[['id','name']])

In [17]:
buf1

,0,id,name
0,"POLYGON ((440907.435 4474302.375, 440905.027 4...",1,Puerta del Sol A
1,"POLYGON ((440907.654 4474278.751, 440905.247 4...",2,Puerta del Sol B
2,"POLYGON ((440620.617 4474679.297, 440618.210 4...",3,Miguel Moya
3,"POLYGON ((440537.988 4475757.290, 440535.581 4...",4,Plaza Conde Suchil
4,"POLYGON ((440903.699 4475561.076, 440901.291 4...",5,Malasaña
...,...,...,...
261,"POLYGON ((438847.647 4477264.758, 438845.239 4...",266,Ciudad Universitaria 1
262,"POLYGON ((438852.434 4477228.086, 438850.026 4...",267,Ciudad Universitaria 2
263,"POLYGON ((438825.418 4477861.052, 438823.010 4...",268,Facultad Biología
264,"POLYGON ((438652.538 4478081.164, 438650.130 4...",269,Facultad Derecho


In [18]:
corte  = buf1.sjoin(carriles_1, predicate='intersects', how='left')

In [19]:
corte.shape, buf.shape 

((463, 4), (266,))

In [20]:
'''
corte = corte.reset_index()

corte['id'] = corte['index'] + 1

'''



"\ncorte = corte.reset_index()\n\ncorte['id'] = corte['index'] + 1\n\n"

In [21]:
corte

,0,id,name,index_right
0,"POLYGON ((440907.435 4474302.375, 440905.027 4...",1,Puerta del Sol A,VÍA USO COMPARTIDO
0,"POLYGON ((440907.435 4474302.375, 440905.027 4...",1,Puerta del Sol A,BICI
1,"POLYGON ((440907.654 4474278.751, 440905.247 4...",2,Puerta del Sol B,VÍA USO COMPARTIDO
1,"POLYGON ((440907.654 4474278.751, 440905.247 4...",2,Puerta del Sol B,BICI
2,"POLYGON ((440620.617 4474679.297, 440618.210 4...",3,Miguel Moya,VÍA USO COMPARTIDO
...,...,...,...,...
261,"POLYGON ((438847.647 4477264.758, 438845.239 4...",266,Ciudad Universitaria 1,BICI
262,"POLYGON ((438852.434 4477228.086, 438850.026 4...",267,Ciudad Universitaria 2,BICI
263,"POLYGON ((438825.418 4477861.052, 438823.010 4...",268,Facultad Biología,BICI
264,"POLYGON ((438652.538 4478081.164, 438650.130 4...",269,Facultad Derecho,BICI


In [22]:
cross=pd.crosstab(index=corte.id , columns=corte.index_right)

In [23]:
cross

index_right,BICI,VÍA USO COMPARTIDO
id,,
1,1,1
2,1,1
3,1,1
4,1,1
5,1,1
...,...,...
266,1,0
267,1,0
268,1,0


In [24]:
cross.reset_index(inplace = True)

In [25]:
cross.set_index('id', inplace = True)

In [26]:
cross

index_right,BICI,VÍA USO COMPARTIDO
id,,
1,1,1
2,1,1
3,1,1
4,1,1
5,1,1
...,...,...
266,1,0
267,1,0
268,1,0


In [27]:
estacionex.head()

,id,number,name,longitude,latitude,address,total_bases,geometry
0,1,1a,Puerta del Sol A,-3.7024255,40.4172137,Puerta del Sol nº 1,30,POINT (440407.435 4474302.375)
1,2,1b,Puerta del Sol B,-3.7024207,40.4170009,Puerta del Sol nº 1,30,POINT (440407.654 4474278.751)
2,3,2,Miguel Moya,-3.7058415,40.4205886,Calle Miguel Moya nº 1,24,POINT (440120.617 4474679.297)
3,4,3,Plaza Conde Suchil,-3.7069171,40.4302937,Plaza del Conde del Valle de Súchil nº 3,18,POINT (440037.988 4475757.290)
4,5,4,Malasaña,-3.7025875,40.4285524,Calle Manuela Malasaña nº 5,24,POINT (440403.699 4475561.076)


In [28]:
estacionex = estacionex.join(cross, on = 'id')

In [29]:
estacionex

,id,number,name,longitude,latitude,address,total_bases,geometry,BICI,VÍA USO COMPARTIDO
0,1,1a,Puerta del Sol A,-3.7024255,40.4172137,Puerta del Sol nº 1,30,POINT (440407.435 4474302.375),1.0,1.0
1,2,1b,Puerta del Sol B,-3.7024207,40.4170009,Puerta del Sol nº 1,30,POINT (440407.654 4474278.751),1.0,1.0
2,3,2,Miguel Moya,-3.7058415,40.4205886,Calle Miguel Moya nº 1,24,POINT (440120.617 4474679.297),1.0,1.0
3,4,3,Plaza Conde Suchil,-3.7069171,40.4302937,Plaza del Conde del Valle de Súchil nº 3,18,POINT (440037.988 4475757.290),1.0,1.0
4,5,4,Malasaña,-3.7025875,40.4285524,Calle Manuela Malasaña nº 5,24,POINT (440403.699 4475561.076),1.0,1.0
...,...,...,...,...,...,...,...,...,...,...
261,266,258,Ciudad Universitaria 1,-3.72699,40.44375,Avenida de la Complutense (Metro Ciudad Univer...,24,POINT (438347.647 4477264.758),1.0,0.0
262,267,259,Ciudad Universitaria 2,-3.72693,40.44342,Avenida de la Complutense (Metro Ciudad Univer...,24,POINT (438352.434 4477228.086),1.0,0.0
263,268,260,Facultad Biología,-3.72731,40.44912,"Jose Antonio Novais, frente al 12",24,POINT (438325.418 4477861.052),1.0,0.0
264,269,261,Facultad Derecho,-3.72937,40.45109,"Avenida Complutense, 23",24,POINT (438152.538 4478081.164),1.0,0.0


In [ ]:
cross.sum()

In [ ]:
corte[corte.isna().any(axis=1)] 



In [ ]:
ax = carriles_1.plot(
    color='blue', figsize = (20,12))

corte.plot(ax=ax, color='red')

plt.show()

In [ ]:
buf = estacionex.buffer(500)

In [ ]:
buf

In [ ]:


carriles = world[['continent', 'geometry']]

continents = world.dissolve(by='continent')

In [ ]:
buf1 = estacionex[estacionex.id == 1].buffer(500)

In [ ]:
buf1.to_frame()

In [ ]:
buf1.to_frame().sjoin(carriles, predicate='intersects')

In [ ]:
ax = ax = carriles.plot(
    color='blue', figsize = (20,12))

buf.plot(ax=ax, color='red', alpha = 0.5)

plt.show()

In [ ]:
ax = barrios.plot(
    color='white', edgecolor='black', figsize = (30,22))

buf.plot(ax=ax, color='red', alpha = 0.5)
estacionex.plot(ax=ax, color='green')


plt.show()